In [1]:
import sympy as sp
import numpy as np
import galois as gf
import pandoc
from math import *

np.set_printoptions(legacy='1.25')

# Homework 2
By Cooper Froemming for EE 4940 - Crytography

## Problem 1

In a finite field GF(p), we can divide a polynomial $a(x)$ by another polynomial $b(x)$ to obtain a quotient $q(x)$ and
a reminder $r(x)$ that satisfy:

(i) $a(x) \stackrel{p}{≡} b(x).q(x) + r(x),$

(ii) $deg(r(x)) < deg(b(x)).$

Here is an example of how it works. Let $a(x) = x^{3} + x + 2,$ $b(x) = 2x + 1$ and $p = 3.$ Note that for a polynomial $m(x),$ we denote the coefficient of $x^{k}$ by $m_{k}.$ For instance, we have ($b_{1}, b_{0}$) = (2, 1):
- Set $q(x) = 0;$
- Find the multiplicative inverse of $b_{1}$ in GF($3$), i.e., $2^{−1}$ $\stackrel{3}{≡} 2$.
- Find the difference between the degree of $a(x)$ and $b(x),$ which is $d = 3 − 1 = 2.$
- Set s(x) = $b^{−1}_{1} ∗ a_{3} ∗ x^{d} = 2x^{2}$ and update $q(x) ← q(x) + s(x) = 2x^{2}$.
- Update $a(x) ← a(x) − b(x) · s(x) = x^{3} + x + 2 − 4x^{3} − 2x^{2} \stackrel{3}{≡} x^{2} + x + 2.$
- Find the difference between the degree of $a(x)$ and $b(x)$, which is $d = 2 − 1 = 1.$
- Set $s(x) = b^{−1}_{1} ∗ a_{2} ∗ x^{d} = 2x$ and update $q(x) ← q(x) + s(x) = 2x^{2} + 2x.$
- Update $a(x) ← a(x) − b(x) · s(x) = (x^{2} + x + 2) − 4x^{2} − 2x \stackrel{3}{≡} 2x + 2.$
- Find the difference between the degree of $a(x)$ and $b(x),$ which is $d = 1 − 1 = 0.$
- Set $s(x) = b^{−1}_{1} ∗ a_{1} ∗ x^{d} = 1$ and update $q(x) ← q(x) + s(x) = 2x^{2} + 2x + 1.$
- Update $a(x) ← a(x) − b(x) · s(x) = (2x + 2) − 2x − 1 \stackrel{3}{≡} 1.$
- Set $r(x) = a(x) = 1,$ since $deg(a(x)) < deg(b(x)).$

Therefore, at the end, we get $q(x) = 2x^{2} + 2x + 1$ and $r(x) = 1.$

(a) Write a code that takes two polynomials $a(x)$ and $b(x),$ together with a prime number $p$ and returns $q(x)$ and $r(x).$ Note that polynomial $m(x) = m_{k}x^{k} + m_{k−1}x^{k−1} + · · · + m_{1}x + m_{0}$ can be represented in the form of the vector of coefficients, i.e., $[m_{k}, m_{k−1}, . . . , m_{2}, m_{1}, m_{0}].$



In [2]:

# Calculate a polynomial
def calc_polynomial(poly_a, input) :
    degree_a = len(poly_a)-1
    i=degree_a
    output = 0
    while i>=0 :
        output = poly_a[degree_a-i]*input**i + output
        i-=1
    return output

#Finds inverse in modulo. e.g. 2 inverse in mod 3 is 2 b/c 2*2 = 4(mod 3) = 1
#==Inputs== 
#input: value to find the inverse of
#modulo: value of the modulus
def find_modulo_inverse(input, modulo) :
    i=modulo
    while i>0 :
        if np.mod(input*i,modulo) == 1 :
            return i
        i-=1
    return 0

#Finds the degree of a polynomial vector by ignoring finding the first non-zero value.
def find_degree(polynomial) :
    shift = 0
    for i in polynomial :
        if i!=0 :
            return len(polynomial)-shift-1
        else : 
            shift +=1
    return 0

#Removes all zeros in a polynomial vector preceeding the first non-zero element.
#e.g. [0, 0, 0, 2, 0, 3, 0] --> [2, 0, 3, 0]
def remove_zeros(polynomial) :
    shift = 0
    for i in polynomial :
        if i!=0 :
            while shift>0 :
                polynomial.pop(0)
                shift-=1
            break
        else : 
            shift +=1
    return polynomial


#Description:   follows the steps outlined in Problem 1 to divide a polynomial a(x) by another polynomial, b(x)
#               to get the quotient q(x) and the remainder r(x)
#==Inputs==
#a: Vector of polynomial coefficients starting at the highest degree. e.g. a(x) 3x^2 + 1 --> poly_a=[3 0 1]
#b: Vector of polynomial coefficients to divide into a(x), same format as a. Note: len(b)<len(a) must be true!
#GF: Galois Field order, e.g. GF(p) --> GF=p, where p must be prime.
#==Outputs
#divide_in_GF[0] = q(x)
#divide_in_GF[1] = r(x)
def divide_in_GF(poly_a, poly_b, GF):
    degree_a = find_degree(poly_a) # Get degree of each polynomial
    print(f"degree a(x): {degree_a}")
    degree_b = find_degree(poly_b)
    print(f"degree b(x): {degree_b}")
    poly_q = 0
    
    #Calculate degree difference between a(x) and b(x)
    if degree_a > degree_b :
        degree_diff = degree_a - degree_b
        print(f"degree difference: {degree_a} - {degree_b} = {degree_diff}")
    else :
        print("ERROR: degree of a < degree of b !")
        return 0

    bk_inverse = find_modulo_inverse(poly_b[0],GF) # find inverse of highest power of b
    print(f"b{degree_b} = {bk_inverse}")

    a_index = 0 #index of poly_a, to be looped over
    poly_s = [0] * (degree_a) #Create an empty matrix of n=degree_diff elements 
    poly_q = [0] * (degree_a)
    while a_index<degree_a :
        print('')
        print(f'STARTLOOP a_index={a_index}')
        print(f'degree_diff = {degree_diff}')


        sq_index = len(poly_s)-1-degree_diff #starts at the first non-zero element and decrements with degree_diff
        print(f'sq_index={sq_index}')
        print(f'a_index={a_index}')

        poly_s[sq_index] = np.mod((bk_inverse*poly_a[a_index]),GF)
        print(f's(x) = {poly_s}')

        poly_q =  np.mod(np.polyadd(poly_s,poly_q),GF)
        print(f'q(x) = {poly_q}')
        print(f's(x)*b(x)={np.mod(np.polymul(poly_s,poly_b),GF)}')

        poly_a = np.mod(np.polysub(poly_a,np.mod(np.polymul(poly_s,poly_b),GF)),GF)
        print(f'a(x) = {poly_a}')
 
        poly_s[sq_index]=0 #reset s(x) = 0 for next loop

        print('ENDLOOP')
        if degree_diff == 0 : #end loop condition.
            break

        #incrementing/decrementing values
        a_index+=1
        sq_index+=1
        degree_diff = find_degree(poly_a) - find_degree(poly_b)
        

    #remove all the preceeding zeros and output answers. 
    remainder = remove_zeros(poly_a.tolist())
    poly_q = remove_zeros(poly_q.tolist())

    return (poly_q,remainder)

#pretty simple function to print polynomial coefficient vectors readably
def print_polynomial(polynomial) :
    degree = len(polynomial)-1
    for i in polynomial :
        print(f'{i}x^{degree} +')
        degree-=1


In [3]:
#The Problem 1 Example
a = [1, 0, 1, 2]
b = [2, 1]
GF = 3

example1 = divide_in_GF(a,b,GF)

degree a(x): 3
degree b(x): 1
degree difference: 3 - 1 = 2
b1 = 2

STARTLOOP a_index=0
degree_diff = 2
sq_index=0
a_index=0
s(x) = [2, 0, 0]
q(x) = [2 0 0]
s(x)*b(x)=[1 2 0 0]
a(x) = [0 1 1 2]
ENDLOOP

STARTLOOP a_index=1
degree_diff = 1
sq_index=1
a_index=1
s(x) = [0, 2, 0]
q(x) = [2 2 0]
s(x)*b(x)=[1 2 0]
a(x) = [0 0 2 2]
ENDLOOP

STARTLOOP a_index=2
degree_diff = 0
sq_index=2
a_index=2
s(x) = [0, 0, 1]
q(x) = [2 2 1]
s(x)*b(x)=[2 1]
a(x) = [0 0 0 1]
ENDLOOP


Let's print out the answer to see if it's correct.

In [4]:
print('q(x) =')
print_polynomial(example1[0])
print('')
print('r(x) =')
print_polynomial(example1[1])

q(x) =
2x^2 +
2x^1 +
1x^0 +

r(x) =
1x^0 +


Yep! That looks like $q(x) = 2x^{2} + 2x + 1$ and $r(x) = 1.$ to me!

(b) Let $a(x) = 5x^{8} + 3x^{3} + 2x^{2} + 4$ and $b(x) = 4x^{3} + x^{2} + 6.$ Compute $q(x)$ and $r(x)$ in GF($7$) using your code.

In [5]:

a = [5, 0, 0, 0, 0, 3, 2, 0, 4]
b = [4, 1, 0, 6]
GF = 7

answer = divide_in_GF(a,b,GF)

degree a(x): 8
degree b(x): 3
degree difference: 8 - 3 = 5
b3 = 2

STARTLOOP a_index=0
degree_diff = 5
sq_index=2
a_index=0
s(x) = [0, 0, 3, 0, 0, 0, 0, 0]
q(x) = [0 0 3 0 0 0 0 0]
s(x)*b(x)=[5 3 0 4 0 0 0 0 0]
a(x) = [0 4 0 3 0 3 2 0 4]
ENDLOOP

STARTLOOP a_index=1
degree_diff = 4
sq_index=3
a_index=1
s(x) = [0, 0, 0, 1, 0, 0, 0, 0]
q(x) = [0 0 3 1 0 0 0 0]
s(x)*b(x)=[4 1 0 6 0 0 0 0]
a(x) = [0 0 6 3 1 3 2 0 4]
ENDLOOP

STARTLOOP a_index=2
degree_diff = 3
sq_index=4
a_index=2
s(x) = [0, 0, 0, 0, 5, 0, 0, 0]
q(x) = [0 0 3 1 5 0 0 0]
s(x)*b(x)=[6 5 0 2 0 0 0]
a(x) = [0 0 0 5 1 1 2 0 4]
ENDLOOP

STARTLOOP a_index=3
degree_diff = 2
sq_index=5
a_index=3
s(x) = [0, 0, 0, 0, 0, 3, 0, 0]
q(x) = [0 0 3 1 5 3 0 0]
s(x)*b(x)=[5 3 0 4 0 0]
a(x) = [0 0 0 0 5 1 5 0 4]
ENDLOOP

STARTLOOP a_index=4
degree_diff = 1
sq_index=6
a_index=4
s(x) = [0, 0, 0, 0, 0, 0, 3, 0]
q(x) = [0 0 3 1 5 3 3 0]
s(x)*b(x)=[5 3 0 4 0]
a(x) = [0 0 0 0 0 5 5 3 4]
ENDLOOP

STARTLOOP a_index=5
degree_diff = 0
sq_index=7
a_inde

In [6]:
print('q(x) =')
print_polynomial(answer[0])
print('')
print('r(x) =')
print_polynomial(answer[1])

q(x) =
3x^5 +
1x^4 +
5x^3 +
3x^2 +
3x^1 +
3x^0 +

r(x) =
2x^2 +
3x^1 +
0x^0 +


## Problem 2

A square matrix is invertible (or full-rank) if and only if its determinant is non-zero or, equivalently, its rows are linearly independent. Let

 $ \begin{align} A = \begin{bmatrix} - & a_1 & - \\ - & a_2 & - \\ - & a_3 & - \\  & \vdots &  \\ - & a_n & - \end{bmatrix} , \end{align} $

be a **full-rank matrix** with its elements from GF($q$), where $a_{i}$ is the *i*th row of matrix A.


(a) How many choices for $a_{1}$ exists?

For an $n \times n$ matrix, there are $(q^{n} - 1)$ choices, because there are $n$ columns, and [0 0 0] is not an option.

(b) Once you fix $a_{1}$, how many choices for $a_{2}$ we have, such that $a_{1}$ and $a_{2}$ are linearly independent?

$(q^{n}-q)$ choices, because for any choice of $a_{1}$, any linear combination of $a_{1}$ is not an option for $a_{2}$, of which there are $p$ linear combinations.

(c) If $a_{1}$ and $a_{2}$ are linearly independent, how linear combination of $a_{1}$ and $a_{2}$ exixts?

$(q^{n}-1)(q^{n}-q)$ choices

(d) Once you fix $a_{1}, a_{2},$ how many choices for $a_{3}$ do we have, such that $a_{3}$ is linearly independent from $(a_{1}, a_{2})?$

For a linear combination of vectors, $v_{1}, v_{2}$, you will have $k_{1}v_{1} + k_{2}v_{2}$, where $k_{1},k_{2} \in q$, there are 

$(q^{n}-q^{2})$ choices.

(e) Given rows $a_{1}, . . . , a_{m−1},$ how many valid choices exist for a general row $a_{m}$ to have a full-rank matrix?

$ (q^{n}-q^{m-1}) $

(f) In general, how many full-rank matrices of size $n$ in GF(q) exists?

$\displaystyle\Pi_{i=0}^{n-1}(q^{n}-q^{i}) $

(g) How many matrices (regardless of their rank) are in GF(q)?

$(q^{n^{n}}) $

(h) If you generate an $n × n$ matrix uniformly at random, what is the probability that it is full-rank?

$\frac {\displaystyle\Pi_{i=0}^{n-1}(q^{n}-q^{i})} {(q^{n^{n}})}$

## Problem 3

Write a computer program that takes a prime number $p$ and an irreducible polynomial $m(x)$ of degree $k$ in GF($p$) to generate GF($p^{k}$) and its operations. Let $α$ be a root of $m(x).$ Note that each element of GF($p^{k}$) is of the form of $b = b_{k−1}α^{k−1} + b_{k−2}α^{k−2} + · · · + b_{1α} + b_{0},$ which you can represent by $B = [b_{k−1}, b_{k−2}, . . . , b_{1}, b_{0}].$ Then, you need to implement two commands

• MyGFAdd($A,B,M,p$) that returns the summation of $a$ and $b$ in GF($p^{k}$);

• MyGFMult($A,B,M,p$) that returns the multiplication of $a$ and $b$ in GF($p^{k}$).

**P.S. Sorry of the documentation is bad on this question, I worked on this problem when I was quite ill the day before this assignment was due.**

### MYGFAdd()

In [7]:
#Finds the degree of a polynomial vector by ignoring finding the first non-zero value.
def find_degree(polynomial) :
    shift = 0
    for i in polynomial :
        if i!=0 :
            return len(polynomial)-shift-1
        else : 
            shift +=1
    return 0

#Removes all zeros in a polynomial vector preceeding the first non-zero element.
#e.g. [0, 0, 0, 2, 0, 3, 0] --> [2, 0, 3, 0]
def remove_zeros(polynomial) :
    shift = 0
    for i in polynomial :
        if i!=0 :
            while shift>0 :
                polynomial.pop(0)
                shift-=1
            break
        else : 
            shift +=1
    return polynomial


# Add A and B modulo p.
#==INPUTS==
#prime: prime number from p in GF(p)
#A: an array of polynomial coefficients e.g. 2x^2 + 1 --> [2 0 1]
#B: same formate as A
#M: do not care
#p: p in GF(p), prime number
#==OUTPUTS==
#Returns polynomial coefficient array that is 
def MyGFAdd(A, B, M, p) :
    degree_A = find_degree(A) #highest degree of A
    degree_B = find_degree(B) #highest degree of B
    degree_diff = abs(degree_A - degree_B)
    A = remove_zeros(A) #remove preceeding zeros
    B = remove_zeros(B)

    #Add A and B differently depending on which one is longer or shorter.
    if degree_A > degree_B : 
        #print('A > B')
        output = [0] * len(A) # A is a longer array
        i=0
        while i<len(A) :
            if i < degree_diff : output[i] = A[i] #don't start adding B until A and B are at the same degree.
            else : output[i] = A[i]+B[i-degree_diff]
            i+=1

    else :
        if degree_A < degree_B : 
            #print('A < B')
            output = [0] * len(B) # B is a logner array
            i=0
            while i<len(B) :
                if i < degree_diff : output[i] = B[i] #don't start adding B until A and B are at the same degree.
                else : output[i] = B[i]+A[i-degree_diff]
                i+=1
        else : #degree_A = degree_B
            #print('A=B')
            output = [0] * len(A)
            i=0
            for element in A :
                output[i] = element+B[i]
                i+=1
    
    # apply modulus to output.
    i=0
    while i<len(output) :
        output[i] = np.mod(output[i], p)
        i+=1

    return output

Testing MyGFAdd():

In [8]:
X = [2, 2, 0, 1]
Y = [2 ,0, 1, 1, 0]
mx = [1, 1, 1, 1]
prime = 3

MyGFAdd(X, Y, mx, prime)

[2, 2, 0, 1, 1]

### MyGFMult()

In [38]:
#Multiplies one value into a full polynomial
#==INPUTS==
#k: scalar coefficent of single value.
#degree_k: degree of 
#A: polynomial coefficient vector.
#==OUTPUTS==
#Returns a polynomial coefficient array: A*kx^(degree)
def single_mult(k,degree_k,A) :
    degree_A = find_degree(A)
    A = remove_zeros(A)
    output = [0] * (degree_A+degree_k+1)

    if k == 0 :
        return output

    i=0
    while i<len(A):
        output[i] = k*A[i]
        i+=1

    return output

#Multiplies two polynomials, ignoring degree > M
def raw_poly_mult_GF(A, B, GF) :
    degree_A = find_degree(A)
    degree_B = find_degree(B)
    
    if degree_A > degree_B : 
        #print('A > B')
        partial_output = [[0 for _ in range(degree_A+degree_B+1)] for _ in range(len(B))]
        i=0
        for element in B :
                partial_output[i] = single_mult(element,(degree_B-i),A)
                i+=1

    else :
        if degree_A < degree_B : 
            #print('A < B')
            partial_output = [[0 for _ in range(degree_A+degree_B+1)] for _ in range(len(A))]
            i=0
            for element in A :
                partial_output[i] = single_mult(element,(degree_A-i),B)
                i+=1

        else : #degree_A = degree_B
            #print('A=B')
            partial_output = [[0 for _ in range(degree_A+degree_B+1)] for _ in range(len(A))]
            i=0
            for element in A :
                partial_output[i] = single_mult(element,(degree_A-i),B)
                i+=1


    max_degree = 0
    for element in partial_output :
        deg = find_degree(element)
        if deg > max_degree :
            max_degree = deg
    #print(f'max_deg={max_degree}')

    output = [0] * (max_degree+1)

    output = partial_output[0]
    #print(f'partial_output={partial_output}')
    i=1
    while i<(len(partial_output)) :
        output = MyGFAdd(output,partial_output[i],0,GF)
        #print(f'output={output}')
        i+=1


    return output

#Finds what the highest degree coefficient is equal to. Assumes that that coefficient = 1.
#e.g. 
# input: [1, 1, 1] output: [2, 2]
#      (x^2 + x + 1)   =   (2x+2)
def find_highest_coefficient(M,p) :
    M[0] = 0
    M = list(np.multiply(M,(-1)))
    M = np.mod(remove_zeros(M),p)
    return M

#Returns the multiplication of A and B in GF(p^k)
#==INPUTS==
#A: coefficient polynomial array A
#B: coefficient polynomial array B
#M: m(x) is an irreducible polynomial in GF(p^k)
#p: prime number p in GF(p)
#==OUTPUTS==
#Returns a polynomial coefficient array of the form Poly [c_n, c_n-1, ..., c1 , c0] --> Poly = c_n*x^n + c_(n-1)*x^(n-1) + ... c1*x + c0
def MyGFMult(A, B, M, p) :
    #find the degree of each polynomial
    degree_A = find_degree(A)
    degree_B = find_degree(B)
    degree_M = find_degree(M)
    degree_diff = abs(degree_A - degree_B)
    #get rid of any preceeding zeros
    A = remove_zeros(A)
    B = remove_zeros(B)
    M = remove_zeros(M)

    #Multiply A and B, still including terms with degree > degree_M
    raw_mult = raw_poly_mult_GF(A,B,p)
    #print(f'rawmult={raw_mult}')
    raw_degree = find_degree(raw_mult)
    inv_M = find_highest_coefficient(M,p) # not actually an inverse, but rather what the highest degree of M equals. e.g. M == x^2 + x + 1, inv_M = 2x+2
    
    #if there needs to be substitutions.
    #e.g. for GF(3), M == x^2 + x + 1, (x^2 = 2x+2), and raw_mult == x^3 --> raw_mult = x(x^2) = x(2x+2) = 2x^2 + 2x = 4x + 4 + 2x = 6x + 4 = 1 (mod 3)
    if raw_degree > degree_M : 
        degree_diff = abs(raw_degree - degree_M)
        
        i=0
        while i<=(len(M)-degree_diff-1) :
            #print(f'Raw_mult1={raw_mult}')
            #print(f'(len(raw_mult)-len(M)+1) = {(len(raw_mult)-len(M)+1)}')
            single_elem = [0] *(len(raw_mult)-len(M)+1)
            single_elem[0] = raw_mult[0]

            raw_mult[0] = 0 #remove highest degree of raw_mult for substitution

            #Debug print statements
            # print(f'Raw_mult2={raw_mult}')
            # print(f'i={i}')
            # print(f'single_elem={single_elem}')
            # print(f'inv_m={inv_M}')
            # print(f'raw_poly_mult_GF(inv_M,single_elem,p)={raw_poly_mult_GF(inv_M,single_elem,p)}')

            #(single_elem*inv_M) is the replacement for the highest degree power of raw_mult, whose result
            #is added back to raw_mult to fully apply the subsitution
            raw_mult = MyGFAdd(raw_poly_mult_GF(inv_M,single_elem,p),raw_mult,M,p) #
            raw_mult = remove_zeros(raw_mult) #important for reducing len(raw_mult) to proper size.
            
            raw_degree = find_degree(raw_mult)
            degree_diff = abs(raw_degree - degree_M)
            
            # print(f'Raw_mult3={raw_mult}')
            # print('')
            
            i+=1

    return raw_mult

    

Test 1

In [41]:
A = [1, 2, 0, 1]
B = [1, 1, 1]
M = [1, 2, 2, 2]
p = 3

#expected value [1, 1, 0]
print(MyGFMult(A,B,M,p))

[1 3 3 3 1 1]
[1, 1, 0]


Test 2

In [40]:
A = [1, 1, 2]
B = [2, 2, 1]
M = [1, 0, 2, 2]
p = 3

#expected value = [2,0]
print(MyGFMult(A,B,M,p))

[2, 0]


## Problem 4

In this problem, you are supposed to decode a ciphertext that is encrypted using the Vig\'enere cipher. You should return the key, the plaintext, and your codes.

You may write all the codes from scratch or use the provided MATLAB codes to simplify your task. The provided files are.

• ciphertext.txt in which the ciphertext is provided.

• EngAlphabetFrequency.mat in which the empirical frequency of the English alphabet is provided. Note that the first character is space, which is followed by 26 letters.

• find rep.m that finds all repeats of a given length in a given text. For instance:
> find rep(’this is the best and simplest scheme.’,4)

> {[14 27]}

as ’est ’ appears in positions 14 and 27 of the sentence.
> find rep(’this is the best and simplest scheme.’,5)

> 0×0 empty cell array

as there is no repeat of length 5 in the sentence.

• EmpiricalFrequency.m that returns the empirical frequency of letters in a text.
Also, plot the histograms of the ciphertext and the plaintext.

In [15]:
import sympy as sp
import numpy as np
import galois as gf
import pandoc
import csv
from math import *

np.set_printoptions(legacy='1.25')

### ProcessFile(filename,type)

In [7]:
#Processes a ciphertext file as according to EmpiricalFrequency.m
def ProcessFile(filename,type) :
    if type == 'file' :
        file = open(filename)
        readfile = file.read()
        input = readfile
    elif type == 'char' :
        input = filename
    else :
        return -1

    alphabet = ' abcdefghijklmnopqrstuvwxyz'
    input = input.lower()

    #Create Known 
    known = [False] * len(input)

    input = list(input)
    #print(input)

    #Replace all unknown characters with spaces
    known = [char if char in alphabet else ' ' for char in input]
    #Replace all double spaces with single spaces
    known_str = ''.join(known)
    known_str_single_space = ' '.join(known_str.split())
    output = list(known_str_single_space)
    return output

### EmpiricalFrequency(filename,type)

In [8]:
def EmpiricalFrequency(filename,type) :
    if type == 'file' :
        file = open(filename)
        readfile = file.read()
        input = readfile
    elif type == 'char' :
        input = filename
    else :
        return -1

    alphabet = ' abcdefghijklmnopqrstuvwxyz'
    input = input.lower()

    #Create Known 
    known = [False] * len(input)

    input = list(input)
    #print(input)

    #Replace all unknown characters with spaces
    known = [char if char in alphabet else ' ' for char in input]
    #Replace all double spaces with single spaces
    known_str = ''.join(known)
    known_str_single_space = ' '.join(known_str.split())

    #Initialize Frequency list
    na=len(alphabet)
    F = np.zeros(na, dtype=int)

    for char in known_str_single_space:
        if char in alphabet:
            ps = alphabet.index(char)  # Get the position of the character in the alphabet
            F[ps] += 1  # Increment the frequency count for this character

    # Normalize the frequency
    F_normalized = F / np.sum(F) if np.sum(F) > 0 else F  # Avoid division by zero
    return F_normalized


### rep.m

In [9]:
#Finds and groups repeats of letters in a string
#==INPUTS==
#A: input string to find frequency of
#K: length of repeats.
#==OUTPUTS==
#Returns the index value of each repeat, grouped into sub-arrays.
def find_rep(A, k):
    n = len(A)
    #print(n)
    pos = []
    all_pos = set()  # Use a set to track all positions
    
    for i in range(0,n - k + 1):
        if i not in all_pos:  # Avoid double reporting
            pattern = A[i:i+k]  # Extract the substring
            occurrences = [j for j in range(n - k + 1) if A[j:j + k] == pattern]
            if len(occurrences) > 1:
                pos.append(occurrences)  # Store the positions
                all_pos.update(occurrences)  # Update the set with new positions
                
    return pos

# Example usage
# result = find_rep('this is the best and simplest scheme.', 4)
# print(result)  # Output: [[13, 26]] (Python indexes at 0, MATLAB indexes at 1)

### find_key_length(input)

In [10]:
# ciphertext_processed = ProcessFile('ciphertext.txt','file')

#Finds the length of a Vignere cipher key.
#==INPUTS==
#input: Vignere ciphertext array where each letter is its own element.
#==OUTPUTS==
#Returns the integer length of a key
def find_key_length(input) :
    repeat_max = [0, []] #format of repeat_max = [<highest number of repeats>, <list of said repeats>]

    for i in find_rep(input,5) :
        if len(i)>repeat_max[0] :
            repeat_max[0] = len(i)
            repeat_max[1] = i

    #print(f'max={repeat_max}')

    differences = [-1] * (len(repeat_max[1])-1)
    i=0
    while i<len(differences) :
        differences[i] = repeat_max[1][i+1]-repeat_max[1][i]
        i+=1

    #print(f'post_diff={differences}')
    key_length = gcd(*differences)
    #print(f'Key Length = {key_length}')

    return key_length

# find_key_length(ciphertext_processed)

# #selected = [806, 1067, 1742, 2111, 2309, 2453, 2579, 3038, 3110]
# #              261,  675,  369,  198,  144,  126,  459,  72,

# #gcd(*[261, 675, 369, 198, 144, 126, 459, 72])

In [11]:
# #Check to see if this works as intended
# for n in range(0,len(grouped_letters)) :
#     print(f'grouped_letters[{n}]={grouped_letters[n]}')

# # expected output:
# # f s y ...
# # q r i ...
# # s e r ...
# # e x t ...
# # w u d ...
# # u p j ...
# # f k a ...
# # j q q ...
# # s d f ...
# #CORRECT!

### decode_cipher(key,filename,alphabet_array)

In [12]:
# Decodes a vignere cipher. 
#==INPUTS==
#key: key that encoded the ciphertext
#filename: filename of the ciphertext
#alphabet_array: array of the form [<string of all alphabet letter>, <dictionary mapping each letter to a number>, <dictionary mapping the number to each letter>, <frequency of each letter in the alphabet>]
#==OUTPUTS==
#Returns plaintext array with each element being a letter of the decoded plaintext.
def decode_cipher(key,filename,alphabet_array) :

    alphabet = alphabet_array[0]
    alphabet_dict = alphabet_array[1]
    inv_alphabet_dict = alphabet_array[2]


    ciphertext = ProcessFile(filename,'file')
    cipher_length = len(ciphertext)
    plaintext = ['?'] * cipher_length
    #print(ciphertext)
    key_length = len(key)
    #print(f'key_length={key_length}')


    #assign number values to letters

    #For each letter, subtract
    


    i=0
    for letter in key :

        for repeat in range(0,(cipher_length//key_length)+1) :
            if (9*repeat+i)<(cipher_length) : #don't leave index range
                cipher_index = 9*repeat+i
            #print(cipher_index)
            plaintext[cipher_index] = inv_alphabet_dict[np.mod((alphabet_dict[ciphertext[cipher_index]] - alphabet_dict[letter]),len(alphabet))]
        i+=1

    return plaintext



### decrypt_vignere_cipher(input_file, alphabet_array)

In [13]:
#==INPUTS==
#input_file: filename of the ciphertext you are trying to decrypt.
#alphabet_array: array of the form [<string of all alphabet letter>, <dictionary mapping each letter to a number>, <dictionary mapping the number to each letter>, <frequency of each letter in the alphabet>]
#==OUTPUTS==
#Returns a 2 element array of [<the decrypted plaintext>, <the key to encode said plaintext>]
def decrypt_vignere_cipher(input_file, alphabet_array) :
    
    #Process the input
    ciphertext_processed = ProcessFile(input_file,'file')
    
    key_length = find_key_length(ciphertext_processed) #length of the key
    num_keys = (len(ciphertext_processed)//key_length)+1 #number of keys divisible into the original ciphertext
    grouped_numbers = [[] * num_keys] * (key_length) # The array that will house the "sets" of letters as shown above.

    alphabet = alphabet_array[0]
    alphabet_frequency = alphabet_array[3]


    #Gather plaintext letters that are "added (mod 7)" by the same key letter into the grouped_numbers array
    #e.g. grouped_numbers = [ 
    #                           [1*key_length+0,            2*key_length+0,             ..., num_keys*key_length+0], 
    #                           [1*key_length+1,            2*key_length+1,             ..., num_keys*key_length+1],
    #                           [1*key_length+(key_length), 2*key_length+(key_length),  ..., num_keys*key_length+(key_length)] 
    #                       ]
    #this would look like: [[0, 8, 17, 26...], [1, 9, 18, 27, ...], ...] for key_length=9
    for n in range(0, key_length) :
        temp = [-1] * num_keys
        for key in range(0, num_keys) :
            #print(f'key={key}')
            #print(f'n={n}')
            value = ((key*key_length)+n)
            if value < len(ciphertext_processed) :
                temp[key] = value
            else :
                break
            #print(f'temp={temp}')
            grouped_numbers[n] = temp
    #print(grouped_numbers)
    #print(f'n={n}')
    #print(f'grouped_numbers[{n}]={grouped_numbers[n]}')


    #Find the associated letter with the index in grouped_numbers
    #e.g. grouped letters = [ [[0,'f'], [8,'s'], ...], [[1,'a'], [9,'b'], ...], ... ]
    i=0
    grouped_letters = [[] * num_keys] * (key_length)
    for b in grouped_numbers :
        grouped_letters
        j=0
        temp = []
        for letter in b :
            temp.append(ciphertext_processed[letter])
        grouped_letters[i] = temp
        i+=1

    #combine all the letters in each pair into one continuous string
    # e.g. [ [[0,'f'], [8,'s'], ...], [[1,'a'], [9,'b'], ...], ... ] --> [['fs'], ['ab'], ...]
    unseparated_grouped_letters = [[] * num_keys] * (key_length)
    for groups in range(0,len(grouped_letters)):
        unseparated_grouped_letters[groups] = ''.join(grouped_letters[groups])


    #find the frequency of every letter each of those continuous strings.
    freq_encoded = [-1] * len(unseparated_grouped_letters)

    index = 0
    for groups in unseparated_grouped_letters :
        freq_encoded[index] = EmpiricalFrequency(groups, 'char')
        # print(EmpiricalFrequency(groups, 'char'))
        index+=1

    paired_freq_encoded = [[[-1, '?'] for _ in range(len(alphabet))] for _ in range(key_length)]
    #print(paired_freq_encoded)

    i=0
    for group in paired_freq_encoded :
        j=0
        #print(group)
        for letter_pair in group :
            paired_freq_encoded[i][j][0] = freq_encoded[i][j]  # Directly modify the original list
            paired_freq_encoded[i][j][1] = alphabet[j]         # Directly modify the origina
            j+=1
        i+=1


    #create a list of all the alphabet letters and their frequencies paired together.
    #e.g. paired_alphabet_frequency = [[' ', 0.185], ['e', 0.064], ..., ['z', 0.00073]]
    paired_alphabet_frequency = [[-1, '?'] for _ in range(len(alphabet))]


    i=0
    for group in paired_alphabet_frequency :
        group[0] = alphabet_frequency[i]
        group[1] = alphabet[i]
        i+=1

    paired_alphabet_frequency.sort(reverse=True)

    #print(f'alphabet frequency = {paired_alphabet_frequency}')

    #sort the encoded letters based on their frequency in the english alphabet.
    for pairs in paired_freq_encoded :
        pairs.sort(reverse=True)


    #Find the Key Length
    key = [] * key_length
    #since ' ' (space) = 0, and its the most frequent character, the most frequent letters the letters of the key.
    for x in paired_freq_encoded :
        key.append(x[0][1])
    key = ''.join(key)
    #print(f'Key = {key}')


    plaintext = ''.join(decode_cipher(key,input_file,alphabet_array))

    return [plaintext, key]





### Testing!

Let's see if it all works!

In [14]:


my_alphabet= ' abcdefghijklmnopqrstuvwxyz'

my_alphabet_dict = {
    ' ': 0,
    'a': 1,
    'b': 2,
    'c': 3,
    'd': 4,
    'e': 5,
    'f': 6,
    'g': 7,
    'h': 8,
    'i': 9,
    'j': 10,
    'k': 11,
    'l': 12,
    'm': 13,
    'n': 14,
    'o': 15,
    'p': 16,
    'q': 17,
    'r': 18,
    's': 19,
    't': 20,
    'u': 21,
    'v': 22,
    'w': 23,
    'x': 24,
    'y': 25,
    'z': 26
}

#The alphabet frequency provided by Prof. Soheil
my_alphabet_frequency = [0.185057120049760, 0.0640553023616335, 0.0145802950629794, 0.0207519961563824, 0.0337670786805122, 0.0975032137896586, 0.0183885252227195, 0.0191935888894409, 0.0497240802504884, 0.0561516595676339, 0.00164143158421966, 0.00830311225500171, 0.0377937580685005, 0.0217067758050807, 0.0552268231153310, 0.0631141332567514, 0.0155745452887443, 0.000913948017250003, 0.0483235552918135, 0.0528674353448129, 0.0692048516144487, 0.0229922916685098, 0.00671544231885557, 0.0179876947073359, 0.000996291807337308, 0.016730760985920, 0.000734288838877702]

my_inv_alphabet_dict = {v: k for k, v in my_alphabet_dict.items()}

my_alphabet_array = [my_alphabet, my_alphabet_dict, my_inv_alphabet_dict, my_alphabet_frequency]

answer = decrypt_vignere_cipher('ciphertext.txt',my_alphabet_array)
print(f'Key = {answer[1]}')
print(f'Plaintext = {answer[0]}')

Key = mineapols
Plaintext = the very first well documented description of a polyalphabetic cipher was by leon battista alberti around year fourteen sixty seven and used a metal cipher disk to switch between cipher alphabets alberti s system only switched alphabets after several words and switches were indicated by writing the letter of the corresponding alphabet in the ciphertext later johannes trithemius in his work polygraphiae invented the tabula recta a critical component of the vigen re cipher the trithemius cipher however provided a progressive rather rigid and predictable system for switching between cipher alphabets in fifteen eighty six blaise de vigenere published a type of polyalphabetic cipher called an autokey cipher because its key is based on the original plaintext before the court of henry iii of france the cipher now known as the vigenere cipher however is based on that originally described by giovan battista bellaso in his book la cifra del sig giovan battista bellaso